### Linear optimization

First we'll get all the probabilities we need: p(x,y|z) for each x,y,z combination:

In [11]:
states = list(product(['treatment','control'],[0,1], [0,1]))

get_conditional_probability = lambda assignment, took, outcome: \
                ( 
                    (df[df.assignment == assignment].took_treatment == took)
                    & 
                    (df[df.assignment == assignment].good_outcome == outcome)  
                ).mean()

# this terrible list comprehension
# collects p(x,y|z) for all 'states'
p_states = {
            f"{assignment}/"
            + f"{'treated' if took == 1 else 'untreated'}/"
            + f"{'good' if outcome ==1 else 'bad'}" : 
                get_conditional_probability(assignment, took, outcome)
                for assignment, took, outcome in states
            }

In [12]:
p_states = {'treatment/untreated/bad': 0.02,
 'treatment/untreated/good': 0.67,
 'treatment/treated/bad': 0.17,
 'treatment/treated/good': 0.14,
 'control/untreated/bad': 0.32,
 'control/untreated/good': 0.04,
 'control/treated/bad': 0.32,
 'control/treated/good': 0.32}

Now we can frame our linear programming problem.

We're maximizing our expression of an ATE given the constraints at hand.

In [13]:
import pulp

# first we set up 'problems', objects that
# can take an objective like 'maximize' as well as constraints

max_problem = pulp.LpProblem("max ATE", pulp.LpMaximize)

We can now specify variables, symbols that the linear program will know to vary in order to maximize or minimize our objective function. These variables are the distribution of **U**: the probabilities of belonging to one of the archetypal (Compliance, Response) partitions.

In [14]:
# Linear optimization ranges over possible variables
# to find those hidden variables that maximize
# or minimize our objective.

# Our variables 'q' are the the probability of 
# being in one of the partitions

partition_names = ['/'.join([compliance, response])
                       for compliance, response in partition_types]

q = {partition: pulp.LpVariable(partition, lowBound=0) for partition in partition_names}

We can add constraints on these variables as well, by simply adding them to the problem we've framed!

In [15]:
# since our vars are probabilities 
# the sum of them should all be under 1

max_problem += sum([v for k,v in q.items()]) == 1

In [16]:
# Let's set the relationship between the distribution of U
# and the probabilities we observe. The variable name
# scheme is p_Z_X_Y


p_treatment_untreated_bad = q['never_taker/never_better'] \
                            + q['defier/never_better'] \
                            + q['never_taker/helped'] \
                            + q['defier/helped'] 

p_treatment_untreated_good = q['never_taker/always_better'] \
                             + q['defier/always_better'] \
                             + q['never_taker/hurt'] \
                             + q['defier/hurt']

p_treatment_treated_bad = q['always_taker/never_better'] \
                          + q['complier/never_better'] \
                          + q['always_taker/hurt'] \
                          + q['complier/hurt']

p_treatment_treated_good = q['always_taker/always_better'] \
                           + q['complier/always_better'] \
                           + q['always_taker/helped'] \
                           + q['complier/helped']

p_control_untreated_bad = q['never_taker/never_better'] \
                          + q['complier/never_better'] \
                          + q['never_taker/helped'] \
                          + q['complier/helped']

p_control_untreated_good = q['never_taker/always_better'] \
                           + q['complier/never_better'] \
                           + q['never_taker/hurt'] \
                           + q['complier/hurt']

p_control_treated_bad = q['always_taker/never_better'] \
                        + q['defier/never_better'] \
                        + q['always_taker/hurt'] \
                        + q['defier/hurt']

p_control_treated_good = q['always_taker/always_better'] \
                         + q['defier/always_better'] \
                         + q['always_taker/helped'] \
                         + q['defier/helped']

In [17]:
# I now apply these linear relationships between U
# and P(X,Y|Z) as constraints on the LP problem.

# Though there's a compact and elegant linear algebraic way
# of doing this, I'm being painfully explicit here because
# it's easier to understand (for me at least)

max_problem += p_treatment_untreated_bad == p_states['treatment/untreated/bad']
max_problem += p_treatment_treated_good == p_states['treatment/treated/good']
max_problem += p_treatment_treated_bad == p_states['treatment/treated/bad']

max_problem += p_control_untreated_bad == p_states['control/untreated/bad']
max_problem += p_control_treated_good == p_states['control/treated/good']
max_problem += p_control_treated_bad == p_states['control/treated/bad']


# I leave some constraints out because it *over*constrains 
# the problem and makes it impossible to solve
# It turns out that the other constraints actually imply 
# these two since they are complimentary probabilities, 
# so we can just leave them commented out

#max_problem += p_control_untreated_good == p_states['control/untreated/good']
#max_problem += p_treatment_untreated_good == p_states['treatment/untreated/good']


With constraints set, all that remains to do set the objective function, our ATE.

$$ATE = P(helped) - P(hurt)$$

where $P(helped)$ is the probability of being in a partition with a 'helped' response type and $P(hurt)$ is the probability of being in a partition with a 'hurt' response type.

Then we can just hit `solve()`.

In [18]:
max_problem += q['complier/helped'] \
               + q['defier/helped'] \
               + q['always_taker/helped'] \
               + q['never_taker/helped'] \
               - q['complier/hurt'] \
               - q['defier/hurt'] \
               - q['always_taker/hurt'] \
               - q['never_taker/hurt']

In [19]:
pulp.LpStatus[max_problem.solve()]

'Optimal'

The problem's been optimized, and now we can see what U looks like in the best case scenario.

In [20]:
q_max = {partition:pulp.value(partition_p) 
         for partition, partition_p in q.items()}

# display

pd.DataFrame(q_max, index=['p(U=u)']).T

,p(U=u)
always_taker/always_better,0.00
always_taker/helped,0.00
always_taker/hurt,0.00
always_taker/never_better,0.01
complier/always_better,0.00
complier/helped,0.14
complier/hurt,0.00
complier/never_better,0.16
defier/always_better,0.32
defier/helped,0.00


Given these we can calculate our best-case ATE.

In [21]:
ate = lambda q: q['complier/helped'] \
               + q['defier/helped'] \
               + q['always_taker/helped'] \
               + q['never_taker/helped'] \
               - q['complier/hurt'] \
               - q['defier/hurt'] \
               - q['always_taker/hurt'] \
               - q['never_taker/hurt']

best_case_ate = ate(q_max)

# display
print("Best-Case ATE: %6.4f" % best_case_ate)

Best-Case ATE: -0.1500
